# **EEG Brain Age – Random Forest Genetic Algorithm + BAG  (Final 2025)**

### Librerias

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
matplotlib.use("Agg")  
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import optuna
import warnings
warnings.filterwarnings("ignore")

c:\Users\Ale\Downloads\Jade\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [2]:
DATA_PATH = r"C:\Users\Ale\Downloads\Jade\Ale\Data_Parametrizada\EEG_features_final.csv"
SAVE_DIR = r"C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_RF_Genetic"
os.makedirs(SAVE_DIR, exist_ok=True)

df = pd.read_csv(DATA_PATH)
df = df.select_dtypes(include=[np.number])
print(f" Dataset loaded: {df.shape[0]} samples, {df.shape[1]} columns")

 Dataset loaded: 510 samples, 10 columns


## **Feature engineering**
Feature engineering consiste en crear nuevas variables a partir de las bandas EEG para representar mejor la actividad cerebral. En este caso, se usan sumas, razones entre bandas y transformaciones matemáticas para capturar relaciones neurofisiológicas relevantes, reducir ruido y facilitar el aprendizaje de los modelos.

In [3]:
if all(col in df.columns for col in ["Delta","Theta","Alpha","Beta","Gamma"]):
    df["TotalPower"] = df[["Delta","Theta","Alpha","Beta","Gamma"]].sum(axis=1)
    df["Theta_Alpha"] = df["Theta"] / (df["Alpha"] + 1e-6)
    df["Alpha_Beta"]  = df["Alpha"] / (df["Beta"] + 1e-6)
    df["Delta_Alpha"] = df["Delta"] / (df["Alpha"] + 1e-6)
    df["Beta_Gamma"]  = df["Beta"]  / (df["Gamma"] + 1e-6)
    df["Slow_Fast"]   = (df["Delta"] + df["Theta"]) / (df["Alpha"] + df["Beta"] + 1e-6)

    for col in ["Delta","Theta","Alpha","Beta","Gamma"]:
        df[f"log_{col}"] = np.log1p(df[col])
        df[f"sqrt_{col}"] = np.sqrt(df[col])

X = df.drop(columns=["Age"])
y = df["Age"]

Z-score filtering was applied to remove outliers, features were standardized to ensure equal scaling, and balanced sample weights were used to reduce bias caused by uneven target distribution. 

In [4]:
z_scores = np.abs(stats.zscore(X))
filtered_entries = (z_scores < 3).all(axis=1)
X = X[filtered_entries].reset_index(drop=True)
y = y[filtered_entries].reset_index(drop=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Balanced sample weights
bins = pd.cut(y, bins=5, labels=False)
weights = 1 / np.bincount(bins)
sample_weights = np.array([weights[b] for b in bins])

# **Training MLP**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)


## **Bayesian hyperparameter optimization using Optuna**

In [8]:
# ===================== IMPORTS =====================
import random
import numpy as np
from deap import base, creator, tools
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# ===================== FITNESS =====================
# ⚠️ Restart kernel before running if already defined
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# ===================== INDIVIDUAL =====================
def random_individual():
    n_estimators = random.randint(200, 800)
    max_depth = random.randint(5, 40)
    min_samples_split = random.randint(2, 10)
    min_samples_leaf = random.randint(1, 5)
    max_features = random.choice(["sqrt", "log2"])

    return creator.Individual([
        n_estimators,
        max_depth,
        min_samples_split,
        min_samples_leaf,
        max_features
    ])

# ===================== BUILD MODEL =====================
def build_ga_rf(individual):
    return RandomForestRegressor(
        n_estimators=individual[0],
        max_depth=individual[1],
        min_samples_split=individual[2],
        min_samples_leaf=individual[3],
        max_features=individual[4],
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )

# ===================== EVALUATE =====================
def evaluate(individual):
    print(
        f"    ▶ Training RF | "
        f"Trees={individual[0]} | "
        f"Depth={individual[1]} | "
        f"Split={individual[2]} | "
        f"Leaf={individual[3]} | "
        f"Feat={individual[4]}"
    )

    model = build_ga_rf(individual)

    model.fit(
        X_train,
        y_train,
        sample_weight=sample_weights[y_train.index]
    )

    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    print(f"    ✔ Finished | R² = {r2:.4f}\n")
    return (r2,)

# ===================== MUTATION =====================
def mutate_individual(individual, indpb=0.2):
    if random.random() < indpb:
        individual[0] = random.randint(200, 800)       # n_estimators
    if random.random() < indpb:
        individual[1] = random.randint(5, 40)          # max_depth
    if random.random() < indpb:
        individual[2] = random.randint(2, 10)          # min_samples_split
    if random.random() < indpb:
        individual[3] = random.randint(1, 5)           # min_samples_leaf
    if random.random() < indpb:
        individual[4] = random.choice(["sqrt", "log2"])# max_features

    return (individual,)

# ===================== TOOLBOX =====================
toolbox = base.Toolbox()
toolbox.register("individual", random_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

# ===================== GA RUN =====================
POP_SIZE = 10
N_GEN = 5
CX_PROB = 0.7
MUT_PROB = 0.2

population = toolbox.population(n=POP_SIZE)

print("\n🔹 Evaluating initial population...\n")
for i, ind in enumerate(population):
    print(f"  Individual {i+1}/{POP_SIZE}")
    ind.fitness.values = toolbox.evaluate(ind)

for gen in range(N_GEN):
    print(f"\n================ GENERATION {gen + 1}/{N_GEN} ================")

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for c1, c2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CX_PROB:
            toolbox.mate(c1, c2)
            del c1.fitness.values, c2.fitness.values

    for mutant in offspring:
        if random.random() < MUT_PROB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid = [ind for ind in offspring if not ind.fitness.valid]
    print(f" Re-evaluating {len(invalid)} offspring...\n")

    for ind in invalid:
        ind.fitness.values = toolbox.evaluate(ind)

    population[:] = offspring

# ===================== BEST RESULT =====================
best_ind = tools.selBest(population, 1)[0]

print("\n================ BEST RF MODEL FOUND ================")
print("n_estimators:", best_ind[0])
print("max_depth:", best_ind[1])
print("min_samples_split:", best_ind[2])
print("min_samples_leaf:", best_ind[3])
print("max_features:", best_ind[4])
print("Best R²:", best_ind.fitness.values[0])




🔹 Evaluating initial population...

  Individual 1/10
    ▶ Training RF | Trees=310 | Depth=38 | Split=10 | Leaf=1 | Feat=log2
    ✔ Finished | R² = 0.7828

  Individual 2/10
    ▶ Training RF | Trees=407 | Depth=23 | Split=8 | Leaf=2 | Feat=sqrt
    ✔ Finished | R² = 0.7836

  Individual 3/10
    ▶ Training RF | Trees=203 | Depth=9 | Split=7 | Leaf=5 | Feat=sqrt
    ✔ Finished | R² = 0.6554

  Individual 4/10
    ▶ Training RF | Trees=702 | Depth=15 | Split=10 | Leaf=4 | Feat=sqrt
    ✔ Finished | R² = 0.6999

  Individual 5/10
    ▶ Training RF | Trees=501 | Depth=40 | Split=10 | Leaf=1 | Feat=log2
    ✔ Finished | R² = 0.7815

  Individual 6/10
    ▶ Training RF | Trees=264 | Depth=38 | Split=9 | Leaf=4 | Feat=log2
    ✔ Finished | R² = 0.6941

  Individual 7/10
    ▶ Training RF | Trees=611 | Depth=21 | Split=7 | Leaf=1 | Feat=sqrt
    ✔ Finished | R² = 0.8417

  Individual 8/10
    ▶ Training RF | Trees=262 | Depth=40 | Split=3 | Leaf=1 | Feat=sqrt
    ✔ Finished | R² = 0.9003

 

### **Train Final Model**

In [11]:

best_params = {
    "n_estimators": best_ind[0],
    "max_depth": best_ind[1],
    "min_samples_split": best_ind[2],
    "min_samples_leaf": best_ind[3],
    "max_features": best_ind[4],
}


print("\nTraining final Random Forest with GA best hyperparameters...")
print(best_params)

rf_best = RandomForestRegressor(
    **best_params,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

# fresh split (important for robustness)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

rf_best.fit(
    X_train,
    y_train,
    sample_weight=sample_weights[y_train.index]
)

y_pred = rf_best.predict(X_test)





Training final Random Forest with GA best hyperparameters...
{'n_estimators': 262, 'max_depth': 40, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}


In [10]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
r = np.corrcoef(y_test, y_pred)[0, 1]

print("\n================ FINAL TEST RESULTS =================")
print(f"MAE (years): {mae:.2f}")
print(f"R²: {r2:.3f}")
print(f"Pearson r: {r:.3f}")


================ FINAL TEST RESULTS =================
MAE (years): 4.52
R²: 0.900
Pearson r: 0.965


### **BAG Calculation and Categorization**

In [12]:
BAG = y_pred - y_test.values
bag_mean, bag_std = BAG.mean(), BAG.std()
print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")

def categorize_bag(bag):
    if bag < -3: return "Resilient"
    elif bag > 3: return "Accelerated"
    else: return "Normal"

bag_categories = np.vectorize(categorize_bag)(BAG)
bag_df = pd.DataFrame({
    "Chronological_Age": y_test.values,
    "Predicted_Age": y_pred,
    "BAG": BAG,
    "Category": bag_categories
})
bag_df.to_csv(os.path.join(SAVE_DIR, "EEG_Brain_Age_Gap_RF.csv"), index=False)


 Brain Age Gap (BAG): mean=-0.74, std=6.63


### Histograms

In [13]:
# BAG histogram
plt.figure(figsize=(7,5))
plt.hist(BAG, bins=25, color="lightcoral", edgecolor="black")
plt.axvline(0, color="blue", linestyle="--")
plt.title("Brain Age Gap (Predicted - Chronological) – Random Forest")
plt.xlabel("BAG (years)")
plt.ylabel("Count")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "bag_hist_RF.png"))
plt.close()

# BAG categories
plt.figure(figsize=(6,4))
pd.Series(bag_categories).value_counts().reindex(["Resilient","Normal","Accelerated"]).plot(kind="bar", color=["#66c2a5","#fc8d62","#8da0cb"])
plt.title("BAG Categories Distribution (±3 years)")
plt.ylabel("Count")
plt.grid(axis="y", alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "bag_categories_RF.png"))
plt.close()



### Feature Importance

In [14]:
feat_imp = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf_best.feature_importances_
}).sort_values("Importance", ascending=False)

feat_imp.to_csv(os.path.join(SAVE_DIR, "RF_feature_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(feat_imp["Feature"][:10], feat_imp["Importance"][:10], color="darkorange")
plt.gca().invert_yaxis()
plt.title("Top 10 Feature Importances – Random Forest")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "RF_feature_importance.png"))
plt.close()

### Permutation Importance

In [15]:

print("\n Computing permutation importance...")
perm_result = permutation_importance(
    rf_best, X_test, y_test, n_repeats=10, random_state=42, scoring="r2"
)
perm_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": perm_result.importances_mean
}).sort_values("Importance", ascending=False)

perm_df.to_csv(os.path.join(SAVE_DIR, "RF_permutation_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(perm_df["Feature"][:10], perm_df["Importance"][:10], color="goldenrod")
plt.gca().invert_yaxis()
plt.title("Top 10 Features – Permutation Importance (RF)")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "RF_permutation_importance.png"))
plt.close()




 Computing permutation importance...


In [16]:

pd.DataFrame([{
    "MAE": mae, "R2": r2, "r": r, "BAG_mean": bag_mean, "BAG_std": bag_std,
    "Best_Params": best_params
}]).to_csv(os.path.join(SAVE_DIR, "RF_final_results.csv"), index=False)

print(f"\n All Random Forest results saved in: {SAVE_DIR}")


 All Random Forest results saved in: C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_RF_Genetic


### Comparison Trials

In [18]:
def BAG_stadistics(y_pred, y_test):
    BAG = y_pred - y_test.values
    bag_mean, bag_std = BAG.mean(), BAG.std()
    print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")
    return bag_mean, bag_std

    
def comparison_trials(n_trials, best_params, X_scaled, y):
    results = []
    for i in range(n_trials):
        print("\nTraining final model with best hyperparameters...")
        print("Trial:", i+1)
        
        rf_best = RandomForestRegressor(
            **best_params,
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled,
            y,
            test_size=0.2,
            random_state=42,
            shuffle=True
        )

        rf_best.fit(
            X_train,
            y_train,
            sample_weight=sample_weights[y_train.index]
        )

        y_pred = rf_best.predict(X_test)

        
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        r = np.corrcoef(y_test, y_pred)[0, 1]
        bag_mean, bag_std = BAG_stadistics(y_pred, y_test)

        results.append({
    "Trial": i + 1,
    "MAE": mae,
    "R2": r2,
    "Pearson_r": r,
    "BAG_mean": bag_mean,
    "BAG_std": bag_std,
    "Model": "RandomForest",
    "N_estimators": best_params["n_estimators"],
    "Max_depth": best_params["max_depth"]
        })


    # save once at the end
    results_df = pd.DataFrame(results)
    results_df.to_csv(
        os.path.join(SAVE_DIR, "TRIALS_RF_Optuna_Genetic.csv"),
        index=False
    )

    print(f"\nAll results saved in: {SAVE_DIR}")

In [19]:
import time
star_time = time.time()
n=20
comparison_trials(20, best_params, X_scaled, y)
end_time = time.time()
print(f"\n Total comparison trials time: {end_time - star_time:.2f} seconds") 


Training final model with best hyperparameters...
Trial: 1

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 2

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 3

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 4

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 5

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 6

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 7

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 8

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 9

 Brain Age Gap (BAG): mean=-0.74, std=6.63

Training final model with best hyperparameters...
Trial: 10

 B

Trials Time: 5,67 seconds